# Thermal models

There are a number of thermal submodels available in PyBaMM. In this notebook we give details of each of the models, and highlight any relevant parameters. At present PyBaMM includes a lumped thermal model, a 1D thermal model which accounts for the through-cell variation in temperature, and a 2D pouch cell model which assumed the temperature is uniform through the thickness of the pouch, but accounts for variations in temperature in the remaining two dimensions. Here we give the governing equations for each model. 

A more comprehensive review of the pouch cell models can be found in references [4] and [6] at the end of this notebook.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


## Lumped models

The lumped thermal model solves the following ordinary differential equation for the average temperature, given here in dimensional terms,

$$
\rho_{eff} \frac{\partial T}{\partial t} = \bar{Q} - \frac{hA}{V}(T-T_{\infty}),
$$

where $\rho_{eff}$ is effective volumetric heat capacity, $T$ is the temperature, $t$ is time, $\bar{Q}$ is the averaged heat source term, $h$ is the heat transfer coefficient, $A$ is the surface area (available for cooling), $V$ is the cell volume, and $T_{\infty}$ is the ambient temperature. An initial temperature $T_0$ must be prescribed.


The effective volumetric heat capacity is computed as 

$$
\rho_{eff} = \frac{\sum_k \rho_k c_{p,k} L_k}{\sum_k L_k},
$$

where $\rho_k$ is the density, $c_{p,k}$ is the specific heat, and $L_k$ is the thickness of each component. The subscript $k \in \{cn, n, s, p, cp\}$ is used to refer to the components negative current collector, negative electrode, separator, positive electrode, and positive current collector.

The heat source term accounts for Ohmic heating $Q_{Ohm,k}$ due to resistance in the solid and electrolyte, irreverisble heating due to electrochemical reactions $Q_{rxn,k}$, and reversible heating due to entropic changes in the the electrode $Q_{rev,k}$:

$$
Q = Q_{Ohm,k}+Q_{rxn,k}+Q_{rev,k},
$$

with

$$ 
Q_{Ohm,k} = -i_k \nabla \phi_k, \quad Q_{rxn,k} = a_k j_k \eta_k, \quad Q_{rev,k} = a_k j_k T_k \frac{\partial U}{\partial T} \bigg|_{T=T_{\infty}}.
$$

Here $i_k$ is the current, $\phi_k$ the potential, $a_k$ the surface area to volume ratio, $j_k$ the interfacial current density, $\eta_k$ the overpotential, and $U$ the open-circuit potential. The averaged heat source term $\bar{Q}$ is computed by taking the volume-average of $Q$.


There are two lumped thermal options in PyBaMM: "lumped" and "x-lumped". Both models solve the same equation, but the term corresponding to heat loss is computed differently.

### "x-lumped" option
The "x-lumped" model assumes a pouch cell geometry in order to compute the overall heat transfer coefficient $h$, surface area $A$, and volume $V$. This model allows the user to select a different heat transfer coefficient on different surfaces of the cell. PyBaMM then automatically computes the overall heat transfer coefficient (see [[1]](#ref), [[2]](#ref2) for details). The parameters used to set the heat transfer coefficients are:

"Negative current collector surface heat transfer coefficient [W.m-2.K-1]"  
"Positive current collector surface heat transfer coefficient [W.m-2.K-1]"  
"Negative tab heat transfer coefficient [W.m-2.K-1]"  
"Positive tab heat transfer coefficient [W.m-2.K-1]"  
"Edge heat transfer coefficient [W.m-2.K-1]"  

and correspond to heat transfer at the large surface of the pouch on the side of the negative current collector, heat transfer at the large surface of the pouch on the side of the positive current collector, heat transfer at the negative tab, heat transfer at the positive tab, and heat transfer at the remaining surfaces.

The "x-lumped" option can be selected as follows


In [2]:
options = {"thermal": "x-lumped"}
model = pybamm.lithium_ion.DFN(options)

### "lumped" option
The behaviour of the "lumped" option changes depending on the "cell geometry" option. By default the "lumped" option sets the "cell geometry" option "arbitrary". This option allows a 1D electrochmical model to be solved, coupled to a lumped thermal model that can be used to model any aribitrary geometry. The user may specify the total heat transfer coefficient $h$, surface area for cooling $A$, and cell volume $V$. The relevant parameters are:  

"Total heat transfer coefficient [W.m-2.K-1]"  
"Cell cooling surface area [m2]"  
"Cell volume [m3]"

which correspond directly to the parameters $h$, $A$ and $V$ in the governing equation.

However, if the "cell geometry" is set to "pouch" the heat transfer coefficient, cell cooling surface are and cell volume are automatically computed to correspond to a pouch. In this instance the "lumped" is equivalent to the "x-lumped" option.

The lumped thermal option with an arbitrary geometry can be selected as follows


In [3]:
# lumped with no geometry option defaults to an arbitrary geometry
options = {"thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options)

# OR 

# lumped with arbitrary geometry specified 
options = {"cell geometry": "arbitrary", "thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options)

The lumped thermal option with a pouch cell geometry can be selected as follows

In [4]:
# lumped with pouch cell geometry (equivalent to choosing "x-lumped" thermal option)
options = {"cell geometry": "pouch", "thermal": "lumped"}
model = pybamm.lithium_ion.DFN(options)

## 1D (through-cell) model

The 1D model solves for $T(x,t)$, capturing variations through the thickness of the cell, but ignoring variations in the other dimensions. The temperature is found as the solution of a partial differential equation, given here in dimensional terms

$$\rho_k c_{p,k} \frac{\partial T}{\partial t} = \lambda_k \nabla^2 T + Q(x,t)$$

with boundary conditions 

$$ -\lambda_{cn} \frac{\partial T}{\partial x}\bigg|_{x=0} = h_{cn}(T_{\infty} - T) \quad -\lambda_{cp} \frac{\partial T}{\partial x}\bigg|_{x=1} = h_{cp}(T-T_{\infty}),$$

and initial condition

$$ T\big|_{t=0} = T_0.$$

Here $\lambda_k$ is the thermal conductivity of component $k$, and the heat transfer coefficients $h_{cn}$ and $h_{cp}$ correspond to heat transfer at the large surface of the pouch on the side of the negative current collector, heat transfer at the large surface of the pouch on the side of the positive current collector, respectively. The heat source term $Q$ is as described in the section on lumped models. Note: the 1D model neglects any cooling from the tabs or edges of the cell -- it assumes a pouch cell geometry and _only_ accounts for cooling from the two large surfaces of the pouch. 

The 1D model is termed "x-full" (since it fully accounts for variation in the x direction) and can be selected as follows


In [5]:
options = {"thermal": "x-full"}
model = pybamm.lithium_ion.DFN(options)

# Pouch cell models

The pouch cell thermal models ignore any variation in temperature through the thickness of the cell (x direction), and solve for $T(y,z,t)$. The temperature is found as the solution of a partial differential equation, given here in dimensional terms,

$$
\rho_{eff} \frac{\partial T}{\partial t} = \lambda_{eff} \nabla_\perp^2T + \bar{Q} - \frac{(h_{cn}+h_{cp})A}{V}(T-T_{\infty}),
$$

along with boundary conditions

$$
-\lambda_{eff} \nabla_\perp T \cdot \boldsymbol{n} = \frac{L_{cn}h_{cn} + (L_n+L_s+L_p+L_{cp})h_{edge}}{L_{cn}+L_n+L_s+L_p+L_{cp}}(T-T_\infty),
$$

at the negative tab,

$$
-\lambda_{eff} \nabla_\perp T \cdot \boldsymbol{n} = \frac{(L_{cn}+L_n+L_s+L_p)h_{edge}+L_{cp}h_{cp}}{L_{cn}+L_n+L_s+L_p+L_{cp}}(T-T_\infty),
$$

at the positive tab, and

$$
-\lambda_{eff} \nabla_\perp T \cdot \boldsymbol{n} = h_{edge}(T-T_\infty),
$$

elsewhere. Again, an initial temperature $T_0$ must be prescribed.

Here the heat source term is averaged in the x direction so that $\bar{Q}=\bar{Q}(y,z)$. The parameter $\lambda_{eff}$ is the effective thermal conductivity, computed as 

$$
\lambda_{eff} = \frac{\sum_k \lambda_k L_k}{\sum_k L_k}.
$$

The heat transfer coefficients $h_{cn}$, $h_{cp}$ and $h_{egde}$ correspond to heat transfer at the large surface of the pouch on the side of the negative current collector, heat transfer at the large surface of the pouch on the side of the positive current collector, and  heat transfer at the remaining, respectively.

As with the "x-lumped" option, the relevant heat transfer parameters are:
"Negative current collector surface heat transfer coefficient [W.m-2.K-1]"
"Positive current collector surface heat transfer coefficient [W.m-2.K-1]"
"Negative tab heat transfer coefficient [W.m-2.K-1]"
"Positive tab heat transfer coefficient [W.m-2.K-1]"
"Edge heat transfer coefficient [W.m-2.K-1]"


# Model comparison

Here we compare the "lumped" thermal model for a pouch cell geometry and an arbitrary geometry. We first set up our models, passing the relevant options

In [6]:
pouch_model = pybamm.lithium_ion.DFN(
    options={"cell geometry": "pouch", "thermal": "lumped"}
)
arbitrary_model = pybamm.lithium_ion.DFN(
    options={"cell geometry": "arbitrary", "thermal": "lumped"}
)

We then pick our parameter set

In [7]:
parameter_values = pybamm.ParameterValues("Marquis2019")

and look at the various parameters related to heat transfer

In [8]:
params = [
    "Negative current collector surface heat transfer coefficient [W.m-2.K-1]",  
    "Positive current collector surface heat transfer coefficient [W.m-2.K-1]",  
    "Negative tab heat transfer coefficient [W.m-2.K-1]",  
    "Positive tab heat transfer coefficient [W.m-2.K-1]",  
    "Edge heat transfer coefficient [W.m-2.K-1]",
    "Total heat transfer coefficient [W.m-2.K-1]",
]

for param in params:
    print(param + ": {}".format(parameter_values[param]))

Negative current collector surface heat transfer coefficient [W.m-2.K-1]: 0.0
Positive current collector surface heat transfer coefficient [W.m-2.K-1]: 0.0
Negative tab heat transfer coefficient [W.m-2.K-1]: 10.0
Positive tab heat transfer coefficient [W.m-2.K-1]: 10.0
Edge heat transfer coefficient [W.m-2.K-1]: 0.3
Total heat transfer coefficient [W.m-2.K-1]: 10.0


We see that the default parameters used for the pouch cell geometry assume that the large surfaces of the pouch are insulated (no heat transfer) and that most of the cooling is via the tabs.

We can also look at the parameters related to the geometry of the pouch

In [9]:
L_cn = parameter_values["Negative current collector thickness [m]"]
L_n = parameter_values["Negative electrode thickness [m]"]
L_s = parameter_values["Separator thickness [m]"]
L_p = parameter_values["Positive electrode thickness [m]"]
L_cp = parameter_values["Positive current collector thickness [m]"]
L_y = parameter_values["Electrode width [m]"]
L_z = parameter_values["Electrode height [m]"]

# total thickness
L = L_cn + L_n + L_s + L_p + L_cp

# compute surface area
A = 2 * (L_y * L_z + L * L_y + L * L_z)
print("Surface area [m2]: {}".format(A))

# compute volume 
V = L * L_y *L_z
print("Volume [m3]: {}".format(V))

Surface area [m2]: 0.056907200000000005
Volume [m3]: 7.798725e-06


and the parameters related to the surface are for cooling and cell volume for the arbitrary geometry 

In [10]:
params = ["Cell cooling surface area [m2]", "Cell volume [m3]"]

for param in params:
    print(param + ": {}".format(parameter_values[param]))

Cell cooling surface area [m2]: 0.0569
Cell volume [m3]: 7.8e-06


We see that both models assume the same cell volume, and the arbitrary model assumes that cooling occurs uniformly from all surfaces of the pouch.

Let's run simulations with both options and compare the results. For demonstration purposes we'll increase the current to amplify the thermal effects

In [11]:
# update current to correspond to a C-rate of 3 (i.e. 3 times the nominal cell capacity)
parameter_values["Current function [A]"] = 3 * parameter_values["Nominal cell capacity [A.h]"]

# pick solver 
solver = pybamm.CasadiSolver(mode="fast", atol=1e-3)

# create simulations in a list
sims = [
    pybamm.Simulation(pouch_model, parameter_values=parameter_values, solver=solver),
    pybamm.Simulation(arbitrary_model, parameter_values=parameter_values, solver=solver)
]

# loop over the list to solve
for sim in sims:
    sim.solve([0, 1000])
    
# plot the results
pybamm.dynamic_plot(
    sims, 
    [
        "Volume-averaged cell temperature [K]", 
        "Volume-averaged total heating [W.m-3]", 
        "Current [A]", 
        "Voltage [V]"
    ],
    labels=["pouch", "arbitrary"],
)    

interactive(children=(FloatSlider(value=0.0, description='t', max=1000.0, step=10.0), Output()), _dom_classes=…

We see that the lumped model with an arbitrary geometry is cooled much more (it is cooled uniformly from all surfaces, compared to the pouch that is only cooled via the tabs and outside edges), which results in the temperature barely changing throughout the simulation. In comparison, the model with the pouch cell geometry is only cooled from a small portion of the overall cell surface, leading to an increase in temperature of around 20 degrees.

## References

The relevant papers for this notebook are:

In [12]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/